In [2]:
TaskObject = "{\"TaskInstanceId\":1,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"TestTask Type Name\", \"Enabled\":1,\"ExecutionUid\":\"8448eabb-9ba4-4779-865b-29e973431273\",\"NumberOfRetries\":0,\"DegreeOfCopyParallelism\":1, \"KeyVaultBaseUrl\":\"https://ads-dev-kv-ads-ic038069.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\", \"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-1,\"EngineName\":\"ads-dev-kv-ads-ic038069\", \"SystemType\":\"Microsoft.Synapse/workspaces\",\"ResourceGroup\":\"sifgofast\",\"SubscriptionId\":\"cd486ba9-eef3-466d-b16c-7f1b2941ae9d\", \"ADFPipeline\":\"GPL_AzureSqlTable_NA_AzureBlobFS_Parquet_Azure\",\"EngineJson\":\"{}\",\"TaskDatafactoryIR\":\"Azure\", \"JsonProperties\":{}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"abfss://adsdevdlsadsic03adsl.dfs.core.windows.net\", \"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"adsdevdlsadsic03\"},\"Instance\":{\"TargetRelativePath\":\"\"}, \"DataFileName\":\"StudentPersonal.json\",\"SourceRelativePath\":\"synapse/sif\",\"SchemaFileName\":\"StudentPersonal.json\",\"Type\":\"Json\"}, \"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://adsdevdlsadsic03adsl.blob.core.windows.net\", \"AuthenticationType\":\"MSI\",\"Type\":\"ABS\",\"Username\":null,\"Container\":\"adsdevdlsadsic03\"}, \"Instance\":{\"TargetRelativePath\":\"synapse/sif\"},\"DataFileName\":\"StudentPersonal\",\"RelativePath\":\"\", \"SchemaFileName\":\"StudentPersonal.json\",\"Type\":\"Delta\"}}"

StatementMeta(adsdevsynspads, 85, 3, Finished, Available)

In [3]:
import random
import json
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

session_id = random.randint(0,1000000)
#invalid source
##TaskObject = "{\"TaskInstanceId\":1,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"TestTask Type Name\",\"Enabled\":1,\"ExecutionUid\":\"8448eabb-9ba4-4779-865b-29e973431273\",\"NumberOfRetries\":0,\"DegreeOfCopyParallelism\":1,\"KeyVaultBaseUrl\":\"https://ark-stg-kv-ads-irud.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\",\"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-1,\"EngineName\":\"ark-stg-adf-ads-irud\",\"SystemType\":\"Datafactory\",\"ResourceGroup\":\"dlzdev04\",\"SubscriptionId\":\"ed1206e0-17c7-4bc2-ad4b-f8d4dab9284f\",\"ADFPipeline\":\"GPL_AzureSqlTable_NA_AzureBlobFS_Parquet_Azure\",\"EngineJson\":\"{}\",\"TaskDatafactoryIR\":\"Azure\",\"JsonProperties\":{}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\"},\"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://arkstgdlsadsirudadsl.dfs.core.windows.net\",\"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"datalakelanding\"},\"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"TestFile.parquet\",\"RelativePath\":\"\",\"SchemaFileName\":\"TestFile.json\",\"Type\":\"Parquet\"}}"
#valid source
#TaskObject = "{\"TaskInstanceId\":1,\"TaskMasterId\":2,\"TaskStatus\":\"InProgress\",\"TaskType\":\"TestTask Type Name\", \"Enabled\":1,\"ExecutionUid\":\"8448eabb-9ba4-4779-865b-29e973431273\",\"NumberOfRetries\":0,\"DegreeOfCopyParallelism\":1, \"KeyVaultBaseUrl\":\"https://ads-dev-kv-ads-ic038069.vault.azure.net/\",\"ScheduleMasterId\":\"-4\",\"TaskGroupConcurrency\":\"10\", \"TaskGroupPriority\":0,\"TaskExecutionType\":\"ADF\",\"ExecutionEngine\":{\"EngineId\":-1,\"EngineName\":\"ads-dev-kv-ads-ic038069\", \"SystemType\":\"Microsoft.Synapse/workspaces\",\"ResourceGroup\":\"sifgofast\",\"SubscriptionId\":\"cd486ba9-eef3-466d-b16c-7f1b2941ae9d\", \"ADFPipeline\":\"GPL_AzureSqlTable_NA_AzureBlobFS_Parquet_Azure\",\"EngineJson\":\"{}\",\"TaskDatafactoryIR\":\"Azure\", \"JsonProperties\":{}},\"Source\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://adsdevdlsadsic03adsl.blob.core.windows.net\", \"AuthenticationType\":\"MSI\",\"Type\":\"ADLS\",\"Username\":null,\"Container\":\"adsdevdlsadsic03\"},\"Instance\":\"\",{\"TargetRelativePath\":\"synapse/sif\"}, \"DataFileName\":\"StudentPersonal.parquet\",\"SourceRelativePath\":\"synapse/sif\",\"SchemaFileName\":\"StudentPersonal.json\",\"Type\":\"Parquet\"}, \"Target\":{\"System\":{\"SystemId\":-8,\"SystemServer\":\"https://adsdevdlsadsic03adsl.blob.core.windows.net\", \"AuthenticationType\":\"MSI\",\"Type\":\"ABS\",\"Username\":null,\"Container\":\"adsdevdlsadsic03\"}, \"Instance\":{\"TargetRelativePath\":\"\"},\"DataFileName\":\"StudentPersonal.parquet\",\"SourceRelativePath\":\"synapse\/sif\", \"SchemaFileName\":\"StudentPersonal.json\",\"Type\":\"Parquet\"}}"
TaskDict = {}
OutputDict = {}
TaskObjectJson = json.loads(TaskObject)

FilewithPath =""

##we want to delete EngineJson as it causes issues when converting back to a json and it is not needed as its properties are within JsonProperties as children
try:
    del TaskObjectJson['ExecutionEngine']['EngineJson']
except:
    print("No EngineJson Found")


testStrings = ['Json', 'Parquet', 'Csv', 'Delta']
try:
    contains = any([test in TaskObjectJson['Source']['Type'] for test in testStrings])
except:
    contains = False


if(contains):
    Source = TaskObjectJson['Source']['System']['Container'] + "@" + TaskObjectJson['Source']['System']['SystemServer'].replace("https://","").replace("blob","dfs") + "/"
    Container = "abfss://" + Source + TaskObjectJson['Source']['SourceRelativePath']
    FilewithPath =  "/" + TaskObjectJson['Source']['SourceRelativePath'] +"/" + TaskObjectJson['Source']['DataFileName'] 
    #+ "."+ TaskObjectJson['Source']['Type']
    Filewithpath = FilewithPath.replace('//', '/')
    
    print(Filewithpath)
    Source = Container  +"/" + TaskObjectJson['Source']['DataFileName'] 
   
    
    print(Source)
else:
    Source = "invalid"


StatementMeta(adsdevsynspads, 85, 4, Finished, Available)

/synapse/sif/StudentPersonal.json
abfss://adsdevdlsadsic03@abfss://adsdevdlsadsic03adsl.dfs.core.windows.net/synapse/sif/StudentPersonal.json

In [16]:
try:
    contains = any([test in TaskObjectJson['Target']['Type'] for test in testStrings])
except:
    contains = False

if(contains):
    Target = TaskObjectJson['Target']['System']['Container'] + "@" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","") + "/"
    TargetFileName = TaskObjectJson['Target']['Instance']['TargetRelativePath'] + "/" + TaskObjectJson['Target']['DataFileName']
    Target = Target + TargetFileName 
    Target = Target.replace('//', '/')
    Target = "abfss://" + Target


else:
    Target = "invalid"
##df = spark.read.load(Source, format=SourceDT)

TaskDict['Source'] = Source
TaskDict['Target'] = Target
TaskDict['SourceType']= TaskObjectJson['Source']['Type'] 
TaskDict['TargetType']= TaskObjectJson['Target']['Type'] 
TaskDict['SourceFileWithPath']=FilewithPath
TaskDict['TargetFileName'] = TargetFileName

SourceHttpPath = TaskObjectJson['Source']['System']['SystemServer'].replace("https://","") + "/" + TaskObjectJson['Source']['System']['Container'] + "/"
SourceHttpPath = SourceHttpPath + TaskObjectJson['Source']['SourceRelativePath'] + "/" + TaskObjectJson['Source']['DataFileName']
SourceHttpPath = SourceHttpPath.replace('//', '/')
SourceHttpPath = "https://" + SourceHttpPath

TargetHttpPath = TaskObjectJson['Target']['System']['Container'] + "/" + TaskObjectJson['Target']['System']['SystemServer'].replace("https://","") + "/"
TargetHttpPath = TargetHttpPath + TaskObjectJson['Target']['Instance']['TargetRelativePath'] + "/" + TaskObjectJson['Target']['DataFileName']
TargetHttpPath = TargetHttpPath.replace('//', '/')
TargetHttpPath = "https://" + TargetHttpPath



OutputDict['SourceHttpPath'] = SourceHttpPath
OutputDict['TargetHttpPath'] = TargetHttpPath


print(TaskDict)
print(OutputDict)

StatementMeta(adsdevsynspads, 85, 17, Finished, Available)

{'Source': 'abfss://adsdevdlsadsic03@abfss://adsdevdlsadsic03adsl.dfs.core.windows.net/synapse/sif/StudentPersonal.json', 'Target': 'abfss://adsdevdlsadsic03@adsdevdlsadsic03adsl.blob.core.windows.net/synapse/sif/StudentPersonal', 'SourceType': 'Json', 'TargetType': 'Delta', 'SourceFileWithPath': '/synapse/sif/StudentPersonal.json', 'TargetFileName': 'synapse/sif/StudentPersonal'}
{'SourceHttpPath': 'https://abfss:/adsdevdlsadsic03adsl.dfs.core.windows.net/adsdevdlsadsic03/synapse/sif/StudentPersonal.json', 'TargetHttpPath': 'https://adsdevdlsadsic03/adsdevdlsadsic03adsl.blob.core.windows.net/synapse/sif/StudentPersonal'}

In [ ]:
#%run /functions

StatementMeta(, , , Cancelled, )

In [5]:
print(Container)
print(Filewithpath)

mssparkutils.fs.ls(Filewithpath)


StatementMeta(adsdevsynspads, 85, 6, Finished, Available)

abfss://adsdevdlsadsic03@abfss://adsdevdlsadsic03adsl.dfs.core.windows.net/synapse/sif
/synapse/sif/StudentPersonal.json


[FileInfo(path=abfss://adsdevdlsadsic03@adsdevdlsadsic03adsl.dfs.core.windows.net/synapse/sif/StudentPersonal.json, name=StudentPersonal.json, size=6321)]

In [7]:
from pyspark.sql.functions import col, explode_outer, from_json, lit, concat
from pyspark.sql.types import StructType, ArrayType
print(Source)
print(FilewithPath)
input_df = spark.read.option("multiline","true").json(FilewithPath)
display(input_df)


#output_df = execute_autoflatten_with_PK(input_df,None)
#display(output_df)



StatementMeta(adsdevsynspads, 85, 8, Finished, Available)

abfss://adsdevdlsadsic03@abfss://adsdevdlsadsic03adsl.dfs.core.windows.net/synapse/sif/StudentPersonal.json
/synapse/sif/StudentPersonal.json


SynapseWidget(Synapse.DataFrame, b12dc3bc-849f-4951-9aa4-8a162f47b34b)

In [9]:
if Container == None: Container = self.TargetRelativePath
 # mssparkutils.fs.put(f"{Container}/{folder}/{filename}", output_df.to_csv(index=False), True) 
print (TargetHttpPath)
print (Target)

#output_df.printSchema()
#schema = output_df.collect()
#loop through each column to find the primary key column

#set up the merge condition used in the next code block
#mergeCondition = "oldData." + primaryKey.COLUMN_NAME + " = newData." + primaryKey.COLUMN_NAME



StatementMeta(adsdevsynspads, 85, 10, Finished, Available)

https://adsdevdlsadsic03/adsdevdlsadsic03adsl.blob.core.windows.net/synapse/sif/StudentPersonal
abfss://adsdevdlsadsic03@adsdevdlsadsic03adsl.blob.core.windows.net/synapse/sif/StudentPersonal

In [13]:
from delta.tables import *

df = input_df

#SIF model uses RefId as Unique identifier in all the structures
mergeCondition = "oldData.RefId = newData.RefId"



if(TaskDict['TargetType'] == 'Delta'):
    print("SourceDT = " + TaskDict['SourceType'] + ", TargetDT = " + TaskDict['TargetType'])
    #df = spark.read.load(Source, format=SourceDT)
    
    var_check = DeltaTable.isDeltaTable(spark, TaskDict['Target'] )
    try:
        if (var_check):
            print("Performing Merge... on Existing table")

            olddt = DeltaTable.forPath(spark, TaskDict['Target']) 

            olddt.alias("oldData").merge(
                df.alias("newData"),
                mergeCondition) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()
        else:
            print("Creating new Delta Table.")    
            df.write.format("{TaskDict['TargetType']}").option('path', TaskDict['Target']).saveAsTable( TaskDict['TargetFileName'])
    except:
        print("Table does not exist. Creating new Delta Table.")    
        df.write.format("TaskDict['TargetType']").option('path', TaskDict['Target']).saveAsTable(TaskDict['TargetFileName'])
elif(TaskDict['TargetType'] == 'Parquet' and TaskDict['SourceType'] == 'Delta'):
    print("SourceDT = Delta, TargetDT = Parquet.")
    df.write.format("{TaskDict['TargetType']}}").mode("overwrite").save(TaskDict['Target']) 

StatementMeta(adsdevsynspads, 85, 14, Finished, Available)

SourceDT = Json, TargetDT = Delta
Creating new Delta Table.
Table does not exist. Creating new Delta Table.

NameError: name 'TargetName' is not defined

In [11]:
TaskDict

StatementMeta(adsdevsynspads, 85, 12, Finished, Available)

{'Source': 'abfss://adsdevdlsadsic03@abfss://adsdevdlsadsic03adsl.dfs.core.windows.net/synapse/sif/StudentPersonal.json',
 'Target': 'abfss://adsdevdlsadsic03@adsdevdlsadsic03adsl.blob.core.windows.net/synapse/sif/StudentPersonal',
 'SourceType': 'Json',
 'TargetType': 'Delta'}